## Project: Personal Chef

"""

1. Take the left over ingredients left over in the kitchen.
2. Searches the web for recipes that use those ingredients and return them.
3. And can answer any follow up questions.

"""

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.jpg', multiple=False)
display(uploader)

FileUpload(value=(), accept='.jpg', description='Upload')

In [3]:
print(uploader.value)

({'name': 'IMG_0916.jpg', 'type': 'image/jpeg', 'size': 1489060, 'content': <memory at 0x104ca9480>, 'last_modified': datetime.datetime(2025, 12, 21, 20, 6, 15, 754000, tzinfo=datetime.timezone.utc)},)


In [4]:
import base64

# Get the first (and only) uploaded file dict
uploaded_file = uploader.value[0]

# This is a memoryview
content_mv = uploaded_file["content"]

# Convert memoryview -> bytes
img_bytes = bytes(content_mv)  # or content_mv.tobytes()

# Now base64 encode
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [5]:
from tavily import TavilyClient
from typing import Dict, Any
from langchain.tools import tool
tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:
    """ Performs web search on ingredients from image """
    
    tavily_client.search(query)

In [6]:
from langchain.messages import HumanMessage

from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model = "gpt-5-nano",
    system_prompt = "Searches the web for recipes that use those ingredients and return them. Also, please be prepared to answer any follow up questions.",
    checkpointer = InMemorySaver(),
    tools = [web_search]
)

config = {"configurable": {"thread_id": "1"}}


multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Tell me how can I use the ingredients present in this image, meaning for which recipes."},
    {"type": "image", "base64": img_b64, "mime_type": "image/jpg"}
])

response = agent.invoke(
    {"messages": [multimodal_question]},
    config
)

full_content = response['messages'][-1].content
print(full_content[:1000])  # prints only the first 1000 characters
print("... [truncated]")


Nice! Based on what I can see in the image, you have:

- Eggs (two cartons)
- Milk (a jug)
- Almonds (in a plastic container)
- Chocolate candies (Hershey’s Pot of Gold and other chocolates)

Here are recipe ideas that use those ingredients (plus a few common pantry items you might have). I’ve kept the ideas practical for what you likely have on hand.

1) Quick chocolate almond mug cake (very fast)
- What you need: eggs, milk, chocolate, almonds (optional for crunch)
- How: melt chocolate with a splash of milk, whisk in 1 egg and a tablespoon of milk, stir in a few crushed almonds if you like, microwave in a mug for 1–2 minutes until set.

2) Chocolate almond smoothie or hot chocolate
- What you need: milk, almonds, chocolate
- How: blend 1 cup milk with a handful of almonds (soaked if you can), and melted chocolate or a scoop of cocoa/cresh chocolate. For a hot version, warm gently on the stove or in the microwave.

3) Homemade almond milk (and a chocolate twist)
- What you need: almo

In [ ]:
question = HumanMessage(content = "What is the Almond-Crusted Baked Eggs exactly? Is it to be included with boiled eggs or srambled eggs? Which one is your personal favorite?")
response = agent.invoke(
    {"messages": [question]},
    config
)
# from pprint import pprint
# pprint(response)